## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Install requirements:


In [ ]:
! pip install --upgrade 'ocdskingfishercolab<0.4' ipywidgets plotly psycopg2-binary > pip.log

Import packages and load extensions:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from google.colab.data_table import DataTable
from google.colab.files import download
from ipywidgets import widgets
from ocdskingfishercolab import (
    download_dataframe_as_csv,
    format_thousands,
    render_json,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_dark_mode,
    set_light_mode,
    set_spreadsheet_name,
)

# Load https://pypi.org/project/ipython-sql/
%load_ext sql
# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table

Set the name of the spreadsheet to export results to, when using `save_dataframe_to_sheet()`:

In [ ]:
spreadsheet_name = 'feedback_results'

set_spreadsheet_name(spreadsheet_name)

Configure the notebook environment:

In [ ]:
# Increase max columns so that Pandas DataFrames with many columns are rendered as data tables.
DataTable.max_columns = 50
# Remove the index from data tables for easier copy-pasting to Google Docs.
DataTable.include_index = False

# Return Pandas DataFrames instead of regular result sets.
%config SqlMagic.autopandas = True
# Don't print number of rows affected.
%config SqlMagic.feedback = False

# If you set Tools > Settings > Site > Theme to dark, uncomment this line.
# set_dark_mode()
# If you are creating plots to copy-paste into reports, uncomment this line.
# set_light_mode()

## Setup Kingfisher Process

### Connect to the database

In [ ]:
import getpass
from ocdskingfishercolab import (
    list_collections,
    list_source_ids,
    set_search_path,
)

Enter your PostgreSQL credentials and connect to the Kingfisher Process database:

In [ ]:
user = input('Username:')
password = getpass.getpass('Password:')

# Don't show connection string after execute.
%config SqlMagic.displaycon = False

connection_string = 'postgresql://' + user + ':' + password + '@postgres.kingfisher.open-contracting.org/kingfisher_process?sslmode=require'
%sql $connection_string

### Choose collections and schema

*Use this section to choose the collections and schema that you want to query.*

#### Set the collection(s)

Update `collection_ids` with the `id`(s) of the [Kingfisher Process collection(s)](https://kingfisher-process.readthedocs.io/en/latest/data-model.html#collections):

In [ ]:
collection_ids = (2358, 2359)

If you don't know which collections you need, run the next cell and use the **Filter** button to filter the [collection table](https://kingfisher-process.readthedocs.io/en/latest/database-structure.html#collection-table) to find the collection(s). You can use the `source_id` column to filter on the `name` of the [Kingfisher Collect spider](https://kingfisher-collect.readthedocs.io/en/latest/spiders.html) used to collect the data. Use the value(s) from the `id` column to update the previous cell.

In [ ]:
list_collections()

#### Set the schema

Update `schema_name` with the name of the [Kingfisher Summarize schema](https://kingfisher-summarize.readthedocs.io/en/latest/index.html#how-it-works).

In [ ]:
schema_name = 'view_data_collection_2358_2359'
set_search_path(schema_name)

If you don't know which schema you need, run the next cell and use the **Filter** button to filter the [selected collections table](https://kingfisher-summarize.readthedocs.io/en/latest/database.html#summaries-selected-collections) to find the schema. You can use the `collection_id` column to filter on the `id` of the collections that you identified in the previous step. Alternatively, you can filter on the `source_id` column. Use the value from the `schema` column to update the previous cell.

In [ ]:
%%sql

SELECT
    summaries.selected_collections.*,
    source_id
FROM
    summaries.selected_collections
    JOIN collection ON summaries.selected_collections.collection_id = collection.id


If you can't find a schema containing the collections that you want to query, you can create a schema using [Kingfisher Summarize](https://ocdsdeploy.readthedocs.io/en/latest/use/kingfisher-summarize.html).

## Check for data collection and processing errors

Kingfisher Collect and Kingfisher Process log messages that might indicate OCDS implementation errors or API stability issues.

Confirm any errors or warnings by manually checking the OCDS publication.

### Kingfisher Collect

See [how to review the Kingfisher Collect crawl’s log file](https://kingfisher-collect.readthedocs.io/en/latest/logs.html).

**Note:** There is an open [pull request](https://github.com/open-contracting/notebooks-ocds/pull/44) to automate this. If interested, please comment on the pull request to prioritize it.

### Kingfisher Process

Users add a note when starting a crawl and when running the `load` command.

Kingfisher Process adds notes (the `note` column) at different levels (the `code` column):

- `INFO`
  - *load*: The Kingfisher Collect crawl's reason for closing and statistics (the `data` column).
- `WARNING`
  - *compile*: When a record has undated releases, linked releases or no releases, but a compiled release can be calculated (by merging remaining dated releases, using the `compiledRelease` field, or using an undated release with a 'compiled' tag).
- `ERROR`
  - *load*: When the input data is invalid JSON.
  - *load*: When Kingfisher Collect yielded a `FileError` item.
  - *compile*: When no compiled release can be calculated for a record.

**Note:** `FileError` items in Kingfisher Process are [scheduled for removal](https://github.com/open-contracting/kingfisher-process/issues/366), as they also appear in the crawl's log file.

List the notes for each of your collections:

In [ ]:
%%sql

SELECT
    collection_id,
    code,
    note,
    data
FROM
    collection_note
WHERE
    collection_id IN :collection_ids


## Check scope



Use this section to check:

* how many releases, records and compiled releases your data contains
* what stages of the contracting process your data covers
* what date range your data covers

If you are preparing an [Ad-hoc structure and format feedback](https://docs.google.com/document/d/1_k7eA2rI-k5EH8VESkVAB73wa_qrpplL-7dKgMLTGZc/edit#heading=h.i7tpu8c49dcv), you might skip this section.

### Release and record counts

Collections in Kingfisher Process contain either [releases](https://standard.open-contracting.org/latest/en/schema/reference/), [records](https://standard.open-contracting.org/latest/en/schema/records_reference/) or [compiled releases](https://standard.open-contracting.org/latest/en/schema/records_reference/#compiled-release). Kingfisher Process creates compiled release collections from release or record collections.

Use this section to check that the data contains the expected number of releases, records and compiled releases. Where possible, you should check these numbers against the total number of results available in the frontend of the data source.

Count the number of releases, records and compiled releases, for each collection.

**Note:** These columns are not yet populated in version 2 of Kingfisher Process. Comment on [this issue](https://github.com/open-contracting/kingfisher-process/issues/370) to prioritize it.

In [ ]:
%%sql

SELECT
    id AS collection_id,
    cached_releases_count AS releases_count,
    cached_records_count AS records_count,
    cached_compiled_releases_count AS compiled_releases_count
FROM
    collection
WHERE
    id IN :collection_ids


### Contracting process stages

Use this section to check that the data covers the expected stages of the contracting process.

#### Release tags

[Release tags](https://standard.open-contracting.org/latest/en/schema/codelists/#release-tag) indicate the stage of a contracting process to which a release is related.

Count the number of releases, for each release tag:

In [ ]:
%%sql

SELECT
    collection_id,
    release_type,
    tag,
    count(*)
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    tag
ORDER BY
    collection_id


#### Objects per stage

In OCDS, data is organized into objects, for each stage of a contracting process. Each compiled release has: at most one `Planning` object, at most one `Tender` object, any number of `Award` objects, and any number of `Contract` objects. Each `Contract` object has at most one `Implementation` object. As such, the number of `Award` objects can exceed the number of unique OCIDs, but the number of `Tender` objects can't.

Plot a count of objects per stage:

In [ ]:
%%sql objects_per_stage <<

SELECT
    CASE WHEN paths.path = 'contracts/implementation' THEN
        'implementation'
    ELSE
        paths.path
    END AS stage,
    CASE WHEN paths.path IN ('planning', 'tender', 'contracts/implementation') THEN
        GREATEST (object_property, 0)
    ELSE
        GREATEST (array_count, 0)
    END AS object_count
FROM (
    SELECT
        unnest(ARRAY['planning', 'tender', 'awards', 'contracts', 'contracts/implementation']) AS path) AS paths
    LEFT JOIN (
        SELECT
            *
        FROM
            field_counts
        WHERE
            collection_id IN :collection_ids
            AND release_type = 'compiled_release'
            AND path IN ('planning', 'tender', 'awards', 'contracts', 'contracts/implementation')) AS field_counts USING (path)


In [ ]:
objects_per_stage_chart = sns.catplot(x="stage", y="object_count", kind="bar", data=objects_per_stage).set_xticklabels(rotation=90)

for ax in objects_per_stage_chart.axes.flat:
  format_thousands(ax.yaxis)

objects_per_stage

### Date ranges


Use this section to check that the data covers the expected date range.

Calculate the earliest and latest `date`, `awards/date` and `contracts/dateSigned`:

In [ ]:
%%sql

SELECT
    collection_id,
    release_type,
    'release_date' AS date_type,
    min(date) AS min,
    max(date) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    date_type
UNION ALL
SELECT
    collection_id,
    release_type,
    'award_date' AS date_type,
    min(first_award_date) AS min,
    max(last_award_date) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type,
    date_type
UNION ALL
SELECT
    collection_id,
    release_type,
    'contract_datesigned' AS date_type,
    min(first_contract_datesigned) AS min,
    max(last_contract_datesigned) AS max
FROM
    release_summary
GROUP BY
    collection_id,
    release_type
ORDER BY
    collection_id,
    release_type,
    date_type;



### Release date distribution

Use this section to check that releases are distributed as expected.

Plot the count of releases per month:

In [ ]:
%%sql release_dates <<

SELECT
    collection_id::text,
    release_type,
    date,
    count(*) AS release_count
FROM
    release_summary rs
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id,
    release_type,
    date
ORDER BY
    date ASC;



In [ ]:
# Resample by month
release_dates = release_dates.set_index('date')
release_dates = release_dates.groupby(['collection_id', 'release_type']).resample("M").sum()

fig, ax = plt.subplots(figsize = [15,5])
sns.lineplot(data = release_dates, x='date', y='release_count', hue = 'collection_id', style = 'release_type')

format_thousands(ax.yaxis)
sns.despine()

### Extensions

Use this section to check which extensions the data uses.

List the extensions declared in the package metadata:

In [ ]:
%%sql

SELECT
    collection_id,
    release_type,
    jsonb_array_elements(package_data -> 'extensions') AS ocds_extension,
    count(*) AS count
FROM
    release_summary
WHERE
    collection_id IN :collection_ids
    AND package_data IS NOT NULL
GROUP BY
    collection_id,
    release_type,
    ocds_extension
ORDER BY
    collection_id,
    release_type,
    count DESC;



## Check for structure and format errors

Kingfisher Collect reformats data sources as release packages or record packages. Check the `data_type` class attribute of the [spider](https://github.com/open-contracting/kingfisher-collect/tree/main/kingfisher_scrapy/spiders). If it is not 'release_package' or 'record_package', recommend to the publisher to package their data.

Kingfisher Process checks data against the OCDS schema using [lib-cove-ocds](https://github.com/open-contracting/lib-cove-ocds), same as the [OCDS Data Review Tool](https://review.standard.open-contracting.org). For release collections, Kingfisher Process stores check results in the `release_check` table. For record collections, Kingfisher Process stores check results in the `record_check` table.

### Confirm that checks are complete

If a crawl is scheduled using Kingfisher Collect, by default, Kingfisher Process performs structural checks. Checking data is the slowest step. For large collections, it is recommended to skip the `check` step or to collect only a sample. Otherwise, there can be a backlog of data to check.

Use this query to determine whether checks are complete for your collection(s).

If checks are in progress, you should wait for the checks to finish before running the queries in this section.

In [ ]:
%%sql

SELECT
    collection_id,
    'release' AS collection_type,
    CASE WHEN count(release.id) = count(release_check.id) THEN
        'complete'
    WHEN count(release_check.id) = 0 THEN
        'not_started'
    ELSE
        'in_progress'
    END AS check_status,
    count(release_check.id)::text || '/' || count(release.id)::text AS check_progress
FROM
    release_check
    RIGHT JOIN release ON release_check.release_id = release.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id
UNION
SELECT
    collection_id,
    'record' AS collection_type,
    CASE WHEN count(record.id) = count(record_check.id) THEN
        'complete'
    WHEN count(record_check.id) = 0 THEN
        'not_started'
    ELSE
        'in_progress'
    END AS check_status,
    count(record_check.id)::text || '/' || count(record.id)::text AS check_progress
FROM
    record_check
    RIGHT JOIN record ON record_check.record_id = record.id
WHERE
    collection_id IN :collection_ids
GROUP BY
    collection_id;



### Error summary

Summarize the errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_error_summary <<

WITH errors AS (
    SELECT
        collection_id,
        errors ->> 'type' AS error_type,
        LEFT (errors ->> 'description',
            49000) AS error,
        ocid,
        errors ->> 'field' AS field,
        errors ->> 'value' AS value,
        row_number() OVER (PARTITION BY collection_id,
            errors ->> 'type',
        LEFT (errors ->> 'description',
        49000)) AS rownum
FROM
    release_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN RELEASE r ON rc.release_id = r.id
    WHERE
        collection_id IN :collection_ids
    UNION ALL
    SELECT
        collection_id,
        errors ->> 'type' AS error_type,
        LEFT (errors ->> 'description',
            49000) AS error,
        ocid,
        errors ->> 'field' AS field,
        errors ->> 'value' AS value,
        row_number() OVER (PARTITION BY collection_id,
            errors ->> 'type',
        LEFT (errors ->> 'description',
        49000)) AS rownum
FROM
    record_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN record r ON rc.record_id = r.id
    WHERE
        collection_id IN :collection_ids
),
examples AS (
    SELECT
        collection_id,
        error_type,
        field,
        error,
        array_agg(ocid) AS example_ocids,
        array_agg(value) AS example_values
    FROM
        errors
    WHERE
        rownum <= 3
    GROUP BY
        collection_id,
        error_type,
        field,
        error
)
SELECT
    collection_id,
    error_type,
    field,
    error,
    count(*) AS count,
    example_ocids,
    example_values
FROM
    errors
    JOIN examples USING (collection_id, error_type, error)
GROUP BY
    collection_id,
    error_type,
    field,
    error,
    example_ocids,
    example_values;



In [ ]:
structure_and_format_error_summary

### Error details

List all errors from the `release_check` and `record_check` tables.

In [ ]:
%%sql structure_and_format_errors <<

SELECT
    collection_id,
    'release' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    LEFT (errors ->> 'description',
        49000) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    release_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN RELEASE r ON rc.release_id = r.id
WHERE
    collection_id IN :collection_ids
UNION ALL
SELECT
    collection_id,
    'record' AS collection_type,
    errors ->> 'type' AS error_type,
    errors ->> 'field' AS field,
    LEFT (errors ->> 'description',
        49000) AS error,
    ocid,
    errors ->> 'value' AS value
FROM
    record_check rc
    CROSS JOIN jsonb_array_elements(cove_output -> 'validation_errors') AS errors
    JOIN record r ON rc.record_id = r.id
WHERE
    collection_id IN :collection_ids


In [ ]:
structure_and_format_errors